In [2]:
#importing library

import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB,BernoulliNB,CategoricalNB
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,f1_score,recall_score,precision_score,confusion_matrix,accuracy_score,roc_auc_score,roc_curve

In [3]:
pd.set_option('display.max_columns', None)
data=pd.read_excel("https://github.com/SaunakMitra/Project/raw/refs/heads/main/telecom_customer_churn.xlsx");data

,Customer ID,Gender,Age,Married,Number of Dependents,City,Zip Code,Latitude,Longitude,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,9,NaN,Yes,42.39,No,Yes,Cable,16.0,No,Yes,No,Yes,Yes,No,No,Yes,One Year,Yes,Credit Card,65.60,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,9,NaN,Yes,10.69,Yes,Yes,Cable,10.0,No,No,No,No,No,Yes,Yes,No,Month-to-Month,No,Credit Card,-4.00,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,4,Offer E,Yes,33.65,No,Yes,Fiber Optic,30.0,No,No,Yes,No,No,No,No,Yes,Month-to-Month,Yes,Bank Withdrawal,73.90,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,13,Offer D,Yes,27.82,No,Yes,Fiber Optic,4.0,No,Yes,Yes,No,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.00,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,3,NaN,Yes,7.38,No,Yes,Fiber Optic,11.0,No,No,No,Yes,Yes,No,No,Yes,Month-to-Month,Yes,Credit Card,83.90,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,Female,20,No,0,La Mesa,91941,32.759327,-116.997260,0,13,Offer D,Yes,46.68,No,Yes,DSL,59.0,Yes,No,No,Yes,No,No,Yes,Yes,One Year,No,Credit Card,55.15,742.90,0.00,0,606.84,1349.74,Stayed,NaN,NaN
7039,9992-RRAMN,Male,40,Yes,0,Riverbank,95367,37.734971,-120.954271,1,22,Offer D,Yes,16.20,Yes,Yes,Fiber Optic,17.0,No,No,No,No,No,Yes,Yes,Yes,Month-to-Month,Yes,Bank Withdrawal,85.10,1873.70,0.00,0,356.40,2230.10,Churned,Dissatisfaction,Product dissatisfaction
7040,9992-UJOEL,Male,22,No,0,Elk,95432,39.108252,-123.645121,0,2,Offer E,Yes,18.62,No,Yes,DSL,51.0,No,Yes,No,No,No,No,No,Yes,Month-to-Month,Yes,Credit Card,50.30,92.75,0.00,0,37.24,129.99,Stayed,NaN,NaN
7041,9993-LHIEB,Male,21,Yes,0,Solana Beach,92075,33.001813,-117.263628,5,67,Offer A,Yes,2.12,No,Yes,Cable,58.0,Yes,No,Yes,Yes,No,Yes,Yes,Yes,Two Year,No,Credit Card,67.85,4627.65,0.00,0,142.04,4769.69,Stayed,NaN,NaN


In [4]:
#Filling Missing Values

data["Avg Monthly Long Distance Charges"]=data["Avg Monthly Long Distance Charges"].fillna(0)
data["Multiple Lines"]=data["Multiple Lines"].fillna("No")
data["Internet Type"]=data["Internet Type"].fillna("No")
data["Avg Monthly GB Download"]=data["Avg Monthly GB Download"].fillna(0)
data["Online Security"]=data["Online Security"].fillna("No")
data["Online Backup"]=data["Online Backup"].fillna("No")
data["Device Protection Plan"]=data["Device Protection Plan"].fillna("No")
data["Premium Tech Support"]=data["Premium Tech Support"].fillna("No")
data["Streaming TV"]=data["Streaming TV"].fillna("No")
data["Streaming Movies"]=data["Streaming Movies"].fillna("No")
data["Streaming Music"]=data["Streaming Music"].fillna("No")
data["Unlimited Data"]=data["Unlimited Data"].fillna("No")
data["Offer"]=data["Offer"].fillna("No")

In [5]:
# load statmodels functions
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# compute the vif for all given features
def compute_vif(considered_features):

    X =data[['Age','Number of Dependents','Number of Referrals','Tenure in Months','Avg Monthly Long Distance Charges','Total Refunds','Total Extra Data Charges','Total Revenue','Avg Monthly GB Download','Monthly Charge']]
    # the calculation of variance inflation requires a constant
    X['intercept'] = 1

    # create dataframe to store vif values
    vif = pd.DataFrame()
    vif["Variable"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif[vif['Variable']!='intercept']
    return vif

compute_vif(['Age','Number of Dependents','Number of Referrals','Tenure in Months','Avg Monthly Long Distance Charges','Total Refunds','Total Extra Data Charges','Total Revenue','Avg Monthly GB Download','Monthly Charge'])


<ipython-input-5-d4b7430de220>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['intercept'] = 1


,Variable,VIF
0,Age,1.312106
1,Number of Dependents,1.153350
2,Number of Referrals,1.200703
3,Tenure in Months,6.817551
4,Avg Monthly Long Distance Charges,1.329710
5,Total Refunds,1.006580
6,Total Extra Data Charges,1.025215
7,Total Revenue,9.562617
8,Avg Monthly GB Download,1.563301
9,Monthly Charge,2.644322


In [6]:
data.head()

,Customer ID,Gender,Age,Married,Number of Dependents,City,Zip Code,Latitude,Longitude,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,9,No,Yes,42.39,No,Yes,Cable,16.0,No,Yes,No,Yes,Yes,No,No,Yes,One Year,Yes,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,9,No,Yes,10.69,Yes,Yes,Cable,10.0,No,No,No,No,No,Yes,Yes,No,Month-to-Month,No,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,4,Offer E,Yes,33.65,No,Yes,Fiber Optic,30.0,No,No,Yes,No,No,No,No,Yes,Month-to-Month,Yes,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,13,Offer D,Yes,27.82,No,Yes,Fiber Optic,4.0,No,Yes,Yes,No,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,3,No,Yes,7.38,No,Yes,Fiber Optic,11.0,No,No,No,Yes,Yes,No,No,Yes,Month-to-Month,Yes,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability


In [7]:
#Mapping

data['Gender']=data['Gender'].map({'Male':1,'Female':0})
data['Customer Status']=data['Customer Status'].map({'Churned':1,'Stayed':0})
data['Married'] = data['Married'].map({'Yes': 1, 'No': 0})
data['Phone Service'] = data['Phone Service'].map({'Yes': 1, 'No': 0})
data['Multiple Lines']=data['Multiple Lines'].map({'Yes': 1, 'No': 0})
data['Internet Service']=data['Internet Service'].map({'Yes': 1, 'No': 0})
data['Online Backup']=data['Online Backup'].map({'Yes': 1, 'No': 0})
data['Online Security']=data['Online Security'].map({'Yes': 1, 'No': 0})
data['Device Protection Plan']=data['Device Protection Plan'].map({'Yes': 1, 'No': 0})
data['Premium Tech Support']=data['Premium Tech Support'].map({'Yes': 1, 'No': 0})
data['Streaming TV']=data['Streaming TV'].map({'Yes': 1, 'No': 0})
data['Streaming Movies']=data['Streaming Movies'].map({'Yes': 1, 'No': 0})
data['Streaming Music']=data['Streaming Music'].map({'Yes': 1, 'No': 0})
data['Unlimited Data']=data['Unlimited Data'].map({'Yes': 1, 'No': 0})
data['Paperless Billing']=data['Paperless Billing'].map({'Yes': 1, 'No': 0})


In [8]:
#Get dummies
data= pd.get_dummies(data,columns=['Offer','Internet Type','Contract', 'Payment Method'], dtype=int, drop_first=True)

In [9]:
#Dropping irrevant column
data=data.drop(columns=["Churn Category",'Churn Reason','Customer ID',"City","Zip Code","Latitude","Longitude"])


In [10]:
data.head()

,Gender,Age,Married,Number of Dependents,Number of Referrals,Tenure in Months,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Paperless Billing,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Offer_Offer A,Offer_Offer B,Offer_Offer C,Offer_Offer D,Offer_Offer E,Internet Type_DSL,Internet Type_Fiber Optic,Internet Type_No,Contract_One Year,Contract_Two Year,Payment Method_Credit Card,Payment Method_Mailed Check
0,0,37,1,0,2,9,1,42.39,0,1,16.0,0,1,0,1,1,0,0,1,1,65.6,593.30,0.00,0,381.51,974.81,0,0,0,0,0,0,0,0,0,1,0,1,0
1,1,46,0,0,0,9,1,10.69,1,1,10.0,0,0,0,0,0,1,1,0,0,-4.0,542.40,38.33,10,96.21,610.28,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1,50,0,0,0,4,1,33.65,0,1,30.0,0,0,1,0,0,0,0,1,1,73.9,280.85,0.00,0,134.60,415.45,1,0,0,0,0,1,0,1,0,0,0,0,0
3,1,78,1,0,1,13,1,27.82,0,1,4.0,0,1,1,0,1,1,0,1,1,98.0,1237.85,0.00,0,361.66,1599.51,1,0,0,0,1,0,0,1,0,0,0,0,0
4,0,75,1,0,3,3,1,7.38,0,1,11.0,0,0,0,1,1,0,0,1,1,83.9,267.40,0.00,0,22.14,289.54,1,0,0,0,0,0,0,1,0,0,0,1,0


In [ ]:

##....................
##Model Building Using SMOTE
##.....................

In [11]:
#Here im using imblearn pipeline
#insted of pipeline because it handle sampling properly
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [12]:
#dividing input and output
x=data.drop(['Customer Status'],axis=1)
y=data['Customer Status']

In [13]:
from sklearn.preprocessing import StandardScaler
std=StandardScaler()
x_train_std=std.fit_transform(x)

In [14]:
#lOGISTIC REGESSION
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
resampling=SMOTE()
pipeline=Pipeline([('SMOTE',resampling),('LOGISTIC' ,lr)])
scores=cross_val_score(pipeline,x_train_std,y,scoring='f1',cv=5)

import numpy as np
print('Average Score is:',scores.mean())
print('Standard Deviation of score:',np.std(scores))

Average Score is: 0.6873518076007532
Standard Deviation of score: 0.016708705933658568


In [15]:
#Support Vector Machine
from sklearn.svm import SVC
svc=SVC(kernel='rbf',C=0.2)
resampling=SMOTE()
pipeline=Pipeline([('SMOTE',resampling),('SVC' ,svc)])
scores=cross_val_score(pipeline,x_train_std,y,scoring='f1',cv=5)
scores.mean()

0.6820161383709963

In [16]:
#Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB,BernoulliNB,CategoricalNB
nb=GaussianNB()
resampling=SMOTE()
pipeline=Pipeline([('SMOTE',resampling),('BAYES' ,nb)])
scores=cross_val_score(pipeline,x_train_std,y,scoring='f1',cv=5)
scores.mean()

0.656280065746633

In [17]:
#LightGBM

from lightgbm import LGBMClassifier
lgb=LGBMClassifier()
resampling=SMOTE()
pipeline=Pipeline([('SMOTE',resampling),('LightGBM' ,lgb)])
scores=cross_val_score(pipeline,x_train_std,y,scoring='f1',cv=5)
scores.mean()

[LightGBM] [Info] Number of positive: 4139, number of negative: 4139
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002958 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5592
[LightGBM] [Info] Number of data points in the train set: 8278, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 4139, number of negative: 4139
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002011 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5747
[LightGBM] [Info] Number of data points in the train set: 8278, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


0.7073224452669524

In [18]:
#XGBOOST CLASSIFIER
from xgboost import XGBClassifier
xgb=XGBClassifier()
resampling=SMOTE()
pipeline=Pipeline([('SMOTE',resampling),('logistic regression' ,xgb)])
scores=cross_val_score(pipeline,x_train_std,y,scoring='f1',cv=5)
scores.mean()

0.6938035598880683

In [19]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
resampling=SMOTE()
pipeline=Pipeline([('SMOTE',resampling),('Decision Tree' ,dt)])
scores=cross_val_score(pipeline,x_train_std,y,scoring='f1',cv=5)
scores.mean()

0.6019391105912717

In [20]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(max_depth=20, min_samples_leaf=2, min_samples_split=5,n_estimators=200)  #max_depth=20, min_samples_leaf=2, min_samples_split=5,n_estimators=200
resampling=SMOTE()
pipeline=Pipeline([('SMOTE',resampling),('Random Forest' ,rf)])
scores=cross_val_score(pipeline,x_train_std,y,scoring='f1',cv=5)
scores.mean()

0.6994954664622306

In [ ]:
## Model Building Using train test split with stratify
#....................................


#....................................

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=33,stratify=y)

In [22]:
std=StandardScaler()
x_train_std=std.fit_transform(x_train)
x_test_std=std.transform(x_test)

In [26]:
lr=LogisticRegression(C=0.1, penalty='l2', solver='lbfgs')  #['C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs']
lr.fit(x_train_std,y_train)
y_pred=lr.predict_proba(x_test_std)[:,1]
y_pred1=lr.predict(x_test_std)
y_train_pred=lr.predict(x_train_std)

print("test_f1:",f1_score(y_test,y_pred1))
print("train_f1:",f1_score(y_train,y_train_pred))

test_f1: 0.6755162241887905
train_f1: 0.6916406520985084


In [27]:
svc=SVC(C=5, gamma=0.01)  #[c=5,gamma=0.01]
svc.fit(x_train_std,y_train)
y_pred=svc.predict(x_test_std)
y_train_pred=svc.predict(x_train_std)

print("test_f1:",f1_score(y_test,y_pred))
print("train_f1:",f1_score(y_train,y_train_pred))

test_f1: 0.679136690647482
train_f1: 0.744300245527885


In [28]:
nb=GaussianNB()
nb.fit(x_train_std,y_train)
y_pred=nb.predict(x_test_std)
y_train_pred=nb.predict(x_train_std)

print("test_f1:",f1_score(y_test,y_pred))
print("train_f1:",f1_score(y_train,y_train_pred))

test_f1: 0.662870159453303
train_f1: 0.6622734761120264


In [29]:
lgb=LGBMClassifier()
lgb.fit(x_train_std,y_train)
y_pred=lgb.predict(x_test_std)
y_train_pred=lgb.predict(x_train_std)

print("test_f1:",f1_score(y_test,y_pred))
print("train_f1:",f1_score(y_train,y_train_pred))

[LightGBM] [Info] Number of positive: 1495, number of negative: 4139
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001641 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1723
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265353 -> initscore=-1.018328
[LightGBM] [Info] Start training from score -1.018328
test_f1: 0.6976744186046512
train_f1: 0.892066734763364


In [31]:
xgb=XGBClassifier(colsample_bytree= 0.8, gamma= 0.5, learning_rate=0.1, max_depth= 3, n_estimators= 100,subsample= 0.6)   #'colsample_bytree': 0.8, 'gamma': 0.5, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.6
xgb.fit(x_train_std,y_train)
y_pred=xgb.predict(x_test_std)
y_train_pred=xgb.predict(x_train_std)

print("test_f1:",f1_score(y_test,y_pred))
print("train_f1:",f1_score(y_train,y_train_pred))


test_f1: 0.688622754491018
train_f1: 0.7382363192750088


In [32]:
rf=RandomForestClassifier()
rf.fit(x_train_std,y_train)
y_pred=rf.predict(x_test_std)
y_train_pred=rf.predict(x_train_std)

print("test_f1:",f1_score(y_test,y_pred))
print("train_f1:",f1_score(y_train,y_train_pred))


test_f1: 0.6787878787878788
train_f1: 1.0


In [33]:
## TRAIN TEST SPLIT AND SMOTE
#.............................
#.........................

In [34]:
sm = SMOTE(random_state=40)
x_res, y_res = sm.fit_resample(x_train,y_train)
std=StandardScaler()
x_train_std=std.fit_transform(x_res)
x_test_std=std.transform(x_test)

In [39]:
lr=LogisticRegression()
lr.fit(x_train_std,y_res)
y_pred=lr.predict(x_test_std)
y_train_pred=lr.predict(x_train_std)

print('test f1_score:',f1_score(y_test,y_pred))
print('train f1 score:',f1_score(y_res,y_train_pred))

test f1_score: 0.6844106463878327
train f1 score: 0.8631406121730972


In [40]:
rf=RandomForestClassifier()
rf.fit(x_train_std,y_res)
y_pred=rf.predict(x_test_std)
y_train_pred=rf.predict(x_train_std)

print('test f1_score:',f1_score(y_test,y_pred))
print('train f1 score:',f1_score(y_res,y_train_pred))

test f1_score: 0.702557200538358
train f1 score: 1.0


In [41]:
svc=SVC(C=5, gamma=0.01)
svc.fit(x_train_std,y_res)
y_pred=svc.predict(x_test_std)
y_train_pred=svc.predict(x_train_std)

print('test f1_score:',f1_score(y_test,y_pred))
print('train f1 score:',f1_score(y_res,y_train_pred))

test f1_score: 0.6761658031088082
train f1 score: 0.8974631268436578


In [ ]:
## Grid Search CV
#...........



#...........

In [ ]:
# Hyper Parameter Tuning for Support Vector Machine
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C': [0.1, 1, 5],
			'gamma': [1, 0.1, 0.01],
			'kernel': ['rbf','sigmoid','poly']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(x_train_std,y_train)
print(grid.best_estimator_)


In [ ]:
# Hyper Parameter Tuning for RandomForest Classifier
from sklearn.ensemble import RandomForestClassifier

# defining parameter range
param_grid = {
    'n_estimators': [100, 200],  # Number of trees in the forest
    'max_features': ['sqrt', 'log2'],  # Number of features to consider at each split
    'max_depth': [10, 20, 30],   # Maximum depth of the tree
    'min_samples_split': [5, 10],   # Minimum number of samples required to split an internal node
    'min_samples_leaf': [2, 4],     # Minimum number of samples required to be at a leaf node
}

# Creating the GridSearchCV object
grid = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    n_jobs=-1,  # Use all available cores
    verbose=3,
    refit=True
)

# Fitting the grid search to the data
grid.fit(x_train_std, y_train)
print(grid.best_estimator_)


Fitting 5 folds for each of 48 candidates, totalling 240 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomForestClassifier(max_depth=20, min_samples_leaf=2, min_samples_split=5,
                       n_estimators=200)


In [ ]:
## Hyper Parameter Tuning for XGBClassifier
# defining parameter range
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3,5,],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.6, 0.8],
    'gamma': [0.1,0.5],
}

# Create a GridSearchCV object with XGBClassifier
grid = GridSearchCV(XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
                    param_grid,
                    refit=True,
                    verbose=3,
                    n_jobs=-1,
                    cv=5)

# Fit the grid with training data
grid.fit(x_train_std, y_train)

# Print the best parameters and best score
print(f"Best parameters found: {grid.best_params_}")
print(f"Best cross-validation score: {grid.best_score_}")


Fitting 5 folds for each of 96 candidates, totalling 480 fits
Best parameters found: {'colsample_bytree': 0.8, 'gamma': 0.5, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.6}
Best cross-validation score: 0.8477085142497807


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:06:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# defining parameter range
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization strength
    'penalty': ['l2'],  # Regularization type
    'solver': ['lbfgs', 'saga']  # Solvers that support l2 and none penalties
}

# Creating the GridSearchCV object
grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, refit=True, verbose=3)

# Fitting the model
grid.fit(x_train_std, y_train)
print(f"Best parameters found: {grid.best_params_}")
print(f"Best cross-validation score: {grid.best_score_}")


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.862 total time=   0.0s
[CV 2/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.833 total time=   0.0s
[CV 3/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.841 total time=   0.0s
[CV 4/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.813 total time=   0.0s
[CV 5/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.830 total time=   0.0s
[CV 1/5] END ....C=0.1, penalty=l2, solver=saga;, score=0.862 total time=   0.1s
[CV 2/5] END ....C=0.1, penalty=l2, solver=saga;, score=0.833 total time=   0.1s
[CV 3/5] END ....C=0.1, penalty=l2, solver=saga;, score=0.841 total time=   0.1s
[CV 4/5] END ....C=0.1, penalty=l2, solver=saga;, score=0.813 total time=   0.1s
[CV 5/5] END ....C=0.1, penalty=l2, solver=saga;, score=0.830 total time=   0.1s
[CV 1/5] END .....C=1, penalty=l2, solver=lbfgs;, score=0.863 total time=   0.0s
[CV 2/5] END .....C=1, penalty=l2, solver=lbfgs;,